# Import thư viện

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import regex as re
from nltk.corpus import stopwords
from keras.models import model_from_json
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Flatten, Dense, Softmax, Dropout
from sklearn.preprocessing import LabelEncoder
import math

In [3]:
movie = pd.read_csv('..\Crawl Data\Crawl Info film\Movie\movie_info.csv')

In [4]:
movie.isnull().sum().sort_values()

Title                           0
Tomatometer score               0
Tomatometer count               0
Audience count                  0
Tomatometer state               0
Cast                          149
Director                      160
Genre                         822
Original Language            1156
Runtime                      1230
Synopsis                     1269
Audience score               3536
Audience state               3536
Release Date (Streaming)     5203
Producer                     5551
Writer                       7185
Release Date (Theaters)     14144
Distributor                 16439
Rating                      18171
Box Office (Gross USA)      20622
Sound Mix                   24092
Aspect Ratio                28438
View the collection         33333
dtype: int64

# Tiền xử lí dữ liệu

## Drop các cột không dùng đến

In [6]:
movie = movie[['Title', 'Tomatometer score', 'Audience score', 'Tomatometer count',
               'Audience count', 'Genre', 'Synopsis', 'Rating']]

In [9]:
movie.isnull().sum().sort_values()

Title                    0
Tomatometer score        0
Tomatometer count        0
Audience count           0
Genre                  822
Synopsis              1269
Audience score        3536
Rating               18171
dtype: int64

## Làm gọn cột "Rating"

In [14]:
for i in range(len(movie)):
    if pd.isna(movie['Rating'].iloc[i]):
        continue
    else:
        movie['Rating'].iloc[i] = movie['Rating'].iloc[i].split(' ')[0]

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_7684\2474650943.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['Rating'].iloc[i] = movie['Rating'].iloc[i].split(' ')[0]


## Drop các dòng mà cột "Synopsis" có giá trị Null

In [15]:
movie.dropna(axis=0, subset=['Synopsis'], inplace=True)

## Xử lí dữ liệu Null ở cột "Rating" và "Genre" bằng mô hình dự báo

In [16]:
# Hàm tiền xử lí dữ liệu text
def preprocess_text(text):
    text = text.lower() # Chuyển văn bản thành chữ thường
    text = re.sub(r'[^\w\s]',' ',text) # Xóa dấu câu
    text = re.sub("\d+", " ", text) # Xóa chữ số
    stop = stopwords.words('english') # Xóa các từ không có nghĩa (Stop Words)
    text = " ".join(text for text in text.split() if text not in stop)
    return text

In [17]:
def predict_info(movie_synopsis):
    t = vectorizer.transform([preprocess_text(movie_synopsis)])
    pre_class = np.argmax(loaded_model.predict(t.todense()), axis=1)
    return pre_class

### Dự báo "Rating"

In [18]:
rating = movie[['Rating', 'Synopsis']].copy().dropna().reset_index(drop=True)

In [19]:
# Tiền xử lí dữ liệu text trong cột "Synopsis" để đưa vào mô hình
for i in range(len(rating)):
    rating['Synopsis'].iloc[i] = preprocess_text(rating['Synopsis'].iloc[i])

In [20]:
# Load model predict Rating đã train
json_file = open('Predict Rating Model\predict_rating.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("Predict Rating Model\predict_rating.h5")

In [21]:
encoder = LabelEncoder()
labels = encoder.fit_transform(rating['Rating'].values)
rating['label'] = labels

In [22]:
x_train, x_test, y_train, y_test = train_test_split(rating['Synopsis'].values, rating['label'].values, test_size = 0.3, random_state = 0)

vectorizer = TfidfVectorizer()
vectorizer.fit_transform(x_train)

<10961x35933 sparse matrix of type '<class 'numpy.float64'>'
	with 367646 stored elements in Compressed Sparse Row format>

In [23]:
list_pre_rating = []

for syp in movie[movie['Rating'].isnull()]['Synopsis'].values:
    rating_class = predict_info(syp)
    rating = encoder.classes_[rating_class][0]
    list_pre_rating.append(rating)

movie.loc[movie['Rating'].isnull(), 'Rating'] = list_pre_rating

1/1 [==============================] - 0s 28ms/step


### Dự báo "Genre"

In [24]:
list_genre = []
for g in movie['Genre']:
    try:
        g = g.split(', ')
        for sub_g in g:
            if sub_g not in list_genre:
                list_genre.append(sub_g)
    except:
        continue

In [25]:
for g in list_genre:
    movie[g] = 0

for i in range(len(movie)):
    for g in list_genre:
        if not pd.isna(movie['Genre'].iloc[i]) and g in movie['Genre'].iloc[i]:
            movie[g].iloc[i] = 1

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_7684\645188401.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie[g].iloc[i] = 1


In [26]:
genre = movie[['Genre', 'Synopsis'] + list_genre].copy().dropna().reset_index(drop=True)

In [27]:
# Tiền xử lí dữ liệu text trong cột "Synopsis" để đưa vào mô hình
for i in range(len(genre)):
    genre['Synopsis'].iloc[i] = preprocess_text(genre['Synopsis'].iloc[i])

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_7684\819770565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre['Synopsis'].iloc[i] = preprocess_text(genre['Synopsis'].iloc[i])


In [28]:
for gr in list_genre:
    print('Predict:', gr)
    # Chia bộ train và test (80:20)
    x_train, x_test, y_train, y_test = train_test_split(genre['Synopsis'].values, genre[gr].values, test_size = 0.2, random_state = 0)

    # Vectorizer
    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform(x_train)

    # Load model predict Genre đã train
    json_file = open('Predict Genre Model\predict_genre_%s.json'%gr, 'r')

    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights('Predict Genre Model\predict_genre_%s.h5'%gr)

    list_pre_rating = []

    for syp in movie[movie['Genre'].isnull()]['Synopsis'].values:
        t = vectorizer.transform([preprocess_text(syp)])
        pred = np.argmax(model.predict(t.todense()),axis=1)[0]
        list_pre_rating.append(int(pred))

    movie.loc[movie['Genre'].isnull(), gr] = list_pre_rating

Predict: Drama
1/1 [==============================] - 0s 29ms/step
[1, 0, 0, 1, 0, 0, 0, 0, 1, 1]
------------------------------------------------------------------------------------------------------------------------------------------------------
Predict: Comedy
1/1 [==============================] - 0s 31ms/step
[0, 0, 1, 0, 1, 1, 0, 1, 1, 0]
------------------------------------------------------------------------------------------------------------------------------------------------------
Predict: Adventure
1/1 [==============================] - 0s 31ms/step
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------------------------------------------------------------------------------------------------------------------------------
Predict: Mystery & thriller
1/1 [==============================] - 0s 39ms/step
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-----------------------------------------------------------------------------------------------------------------------------------------

### Thay thế giá trị dự đoán vào các dòng có cột "Genre" null

In [29]:
a_list = []

for i in range(len(movie)):
    t = []
    for genre in list_genre:
        g = movie.iloc[i][genre]
        if g == 1:
            t.append(genre)
    a_list.append(', '.join(t))

In [30]:
movie['Genre'] = a_list
movie.drop(columns=list_genre, inplace=True)

## Drop các dòng trùng lặp

In [31]:
movie.duplicated().sum()

11

In [32]:
movie.drop_duplicates(inplace=True)

## Thay thế Null ở cột "Audience score" bằng giá trị ở cột "Tomatometer score" tương ứng

In [ ]:
movie['Audience score'].loc[movie['Audience score'].isnull()] = movie['Tomatometer score'].loc[movie['Audience score'].isnull()]

In [36]:
movie.isnull().sum().sort_values()

Title                           0
Tomatometer score               0
Audience score                  0
Tomatometer count               0
Audience count                  0
Tomatometer state               0
Genre                           0
Original Language               0
Rating                          0
Synopsis                        0
Cast                           23
Director                       80
Runtime                       154
Audience state               2907
Release Date (Streaming)     4070
Producer                     4461
Writer                       6137
Release Date (Theaters)     12919
Distributor                 15208
Box Office (Gross USA)      19359
Sound Mix                   22821
Aspect Ratio                27165
View the collection         32053
dtype: int64

## Tính điểm của phim (trung bình cộng của 2 cột "Tomatometer score" và "Audience score")

In [37]:
movie['Score'] = (movie['Tomatometer score'] + movie['Audience score'])/2

In [38]:
movie.drop(columns=['Tomatometer score', 'Audience score'], inplace=True)

## Tính số lượt review

In [39]:
def get_tomatometer_count(val):
    return int(val.split(' ')[0])

In [40]:
movie['Tomatometer count'] = movie['Tomatometer count'].apply(get_tomatometer_count)

In [41]:
movie['Audience count'].unique()

array(['1,000+ Ratings', 'Fewer than 50 Ratings', '2,500+ Ratings',
       '50,000+ Ratings', '500+ Ratings', '5,000+ Ratings',
       '100,000+ Ratings', '100+ Ratings', '25,000+ Ratings',
       '50+ Ratings', '250,000+ Ratings', '10,000+ Ratings',
       '250+ Ratings', '1,000+ Verified Ratings', '0 Ratings',
       '2,500+ Verified Ratings', '10,000+ Verified Ratings',
       '100+ Verified Ratings', '5,000+ Verified Ratings',
       '50+ Verified Ratings', 'Fewer than 50 Verified Ratings',
       '500+ Verified Ratings', '250+ Verified Ratings',
       '50,000+ Verified Ratings', '25,000+ Verified Ratings', 'Ratings'],
      dtype=object)

In [42]:
def get_audience_count(val):
    if '+' in val:
        kq = int(val.split('+')[0].replace(',', ''))
    else:
        kq = 50
    return kq

In [43]:
movie['Audience count'] = movie['Audience count'].apply(get_audience_count)

In [44]:
movie['Review count'] = movie['Tomatometer count'] + movie['Audience count']

In [45]:
movie.drop(columns=['Tomatometer count', 'Audience count'], inplace=True)

# Lưu dữ liệu đã qua tiền xử lí thành file csv

In [12]:
movie.to_csv('movie_preprocessing5.csv', index=False)